In [24]:
!pip install autocorrect




## Step 1: Preparing the environment

In [25]:
import pandas as pd
import re
import pickle


from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

from nltk.corpus import stopwords
from autocorrect import Speller
import sklearn
import numpy as np
import matplotlib as plt
#import pycaret
import nltk
import spacy

from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, f1_score
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 0.20.1.


In [26]:
pip install simpletransformers

Note: you may need to restart the kernel to use updated packages.


In [27]:
pip install spacy

Note: you may need to restart the kernel to use updated packages.


In [28]:
pip install scikit-learn==0.20.1

Note: you may need to restart the kernel to use updated packages.


In [29]:
import sklearn
print('The scikit-learn version is {}.'.format(sklearn.__version__))

The scikit-learn version is 0.20.1.


## Step 2: Text Pre-processing

In [30]:
#Functions used in this script

# created but can assume these companies spelled it right in there TOA
def spelling(text):
    
    spell = Speller(lang='en')
    return str(spell(text))
    
def tokening(text):
    
    #nltk.download('punkt')
    tokenized_word=word_tokenize(text)
    return tokenized_word
    
def removing_stopwords(text):
    
    #nltk.download('punkt')
    tokenized_word1=tokening(text)
    #nltk.download('stopwords')
    stop_words=set(stopwords.words("english"))
    filtered_sent=[]
    for w in tokenized_word1:
        if w not in stop_words:
            filtered_sent.append(w)
    return " ".join(filtered_sent)



def Lemmating(text):
    #nltk.download('wordnet')
    lemmatizer = WordNetLemmatizer()
    tokenized_word1=tokening(text)
    stemmed_words=[]
    for w in tokenized_word1:
        stemmed_words.append(lemmatizer.lemmatize(w))
    return " ".join(stemmed_words)

def pos_tag(text):
    #nltk.download('averaged_perceptron_tagger')
    tokenized_word1=tokening(text)
    pos=nltk.pos_tag(removing_stopwords(text1))
    return pos
    
# function called for cleaning html tags from text
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext


In [59]:
# import data
data = pd.read_csv('datapoint_33.csv')
data.head()

,CompanyName,Point,QuoteText,QuoteDoc,LaymanTerm,Index Topic
0,scratch,neutral,<p>\n<i>The Scratch Privacy Policy was last up...,Privacy Policy,There is a date of the last update of the terms,Notice of Changing Terms
1,trello,bad,\n</ul>Sharing with third parties <p>We share ...,Privacy Policy,Third parties may be involved in operating the...,Third Parties
2,trello,bad,"Send ""Do Not Track"" Signals:</b> Some browsers...",Privacy Policy,This service ignores the Do Not Track (DNT) he...,Cookies
3,trello,good,You have the right to request a copy of your i...,Trello Privacy Policy,You can request access and deletion of persona...,NaN
4,trello,neutral,These Cloud Terms of Service will go into effe...,Atlassian Cloud Terms of Service,There is a date of the last update of the terms,Notice of Changing Terms


In [60]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2140 entries, 0 to 2139
Data columns (total 6 columns):
CompanyName    2140 non-null object
Point          2140 non-null object
QuoteText      2140 non-null object
QuoteDoc       2140 non-null object
LaymanTerm     2140 non-null object
Index Topic    909 non-null object
dtypes: object(6)
memory usage: 100.4+ KB


In [61]:
data=data.dropna(axis=1,how='all')
data.reset_index(drop=True)

,CompanyName,Point,QuoteText,QuoteDoc,LaymanTerm,Index Topic
0,scratch,neutral,<p>\n<i>The Scratch Privacy Policy was last up...,Privacy Policy,There is a date of the last update of the terms,Notice of Changing Terms
1,trello,bad,\n</ul>Sharing with third parties <p>We share ...,Privacy Policy,Third parties may be involved in operating the...,Third Parties
2,trello,bad,"Send ""Do Not Track"" Signals:</b> Some browsers...",Privacy Policy,This service ignores the Do Not Track (DNT) he...,Cookies
3,trello,good,You have the right to request a copy of your i...,Trello Privacy Policy,You can request access and deletion of persona...,NaN
4,trello,neutral,These Cloud Terms of Service will go into effe...,Atlassian Cloud Terms of Service,There is a date of the last update of the terms,Notice of Changing Terms
...,...,...,...,...,...,...
2135,imdb,blocker,IMDb has the right but not the obligation to m...,Terms of Service,The service can delete specific content withou...,Ownership
2136,imdb,bad,"However, because cookies allow you to take ful...",Privacy Policy,Blocking cookies may limit your ability to use...,NaN
2137,imdb,bad,This Notice and the Terms and Conditions of Us...,Privacy Policy,Terms may be changed any time at their discre...,NaN
2138,imdb,neutral,"Last Updated, February 8, 2018</b>",Privacy Policy,There is a date of the last update of the terms,Notice of Changing Terms


In [62]:
data.isnull().sum()

CompanyName       0
Point             0
QuoteText         0
QuoteDoc          0
LaymanTerm        0
Index Topic    1231
dtype: int64

In [63]:
## Makes a new data frame of data by splitting any phrases into sentences and lining it up with the same label
nltk.download('punkt')
sentences = []
for row in data.itertuples():
    for sentence in sent_tokenize(row[3]):
        if type(sentence) != float:
            if sentence != '':
                sentences.append((row[2], sentence))
data_sentenced = pd.DataFrame(sentences, columns=['Point', 'Sentence'])

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/nayefaboutayoun/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [64]:
# remove any non alpha numeric

data_sentenced['Sentence'] = data_sentenced.apply(lambda x: re.sub('[^0-9a-zA-Z]+', ' ', x['Sentence']), axis=1)
#data_sentenced['Sentence'] = data_sentenced.apply(lambda x: re.sub("^\d+\s|\s\d+\s|\s\d+$", " ", x['Sentence']), axis=1)

In [65]:
#remove all tags like <p> etc. in the data frame "data_sentenced"
data_sentenced['Sentence'] = data_sentenced['Sentence'].str.lower()
data_sentenced['Sentence'] = data_sentenced.apply(lambda x: cleanhtml(x['Sentence']), axis=1)

In [66]:
#remove stop words
nltk.download('stopwords')
data_sentenced['Sentence'] = data_sentenced.apply(lambda x: removing_stopwords(x['Sentence']), axis=1)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/nayefaboutayoun/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [67]:
#Lemmaing words
nltk.download('wordnet')

data_sentenced['Sentence'] = data_sentenced.apply(lambda x: Lemmating(x['Sentence']), axis=1)

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/nayefaboutayoun/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [68]:
data_sentenced.Point.unique()

array(['neutral', 'bad', 'good', 'blocker'], dtype=object)

In [69]:
data_sentenced.describe()

,Point,Sentence
count,3882,3882
unique,4,3797
top,bad,p
freq,1809,17


In [70]:
data_sentenced.describe(include = 'O')

,Point,Sentence
count,3882,3882
unique,4,3797
top,bad,p
freq,1809,17


In [71]:
data_sentenced.head()

,Point,Sentence
0,neutral,p scratch privacy policy last updated may 23 2018
1,bad,ul sharing third party p share information thi...
2,bad,service provider need access information perfo...
3,bad,may also share information third party agreed ...
4,bad,may give third party apps access account infor...


In [72]:
# change label to binary either bad or not bad
data_sentenced['Point'].replace(['blocker', 'neutral', 'good'], 0, inplace=True)
data_sentenced['Point'].replace(['bad'], 1, inplace=True)

data_sentenced.head()

,Point,Sentence
0,0,p scratch privacy policy last updated may 23 2018
1,1,ul sharing third party p share information thi...
2,1,service provider need access information perfo...
3,1,may also share information third party agreed ...
4,1,may give third party apps access account infor...


In [73]:
data_sentenced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3882 entries, 0 to 3881
Data columns (total 2 columns):
Point       3882 non-null int64
Sentence    3882 non-null object
dtypes: int64(1), object(1)
memory usage: 60.8+ KB


# Step 3: Creating Training and Testing Dataset 

In [74]:
#place into series for input and label

X, y = data_sentenced['Sentence'], data_sentenced['Point']

X_train, X_val, y_train, y_val = sklearn.model_selection.train_test_split(X, y, test_size=0.2, random_state=42)

In [75]:
X

0       p scratch privacy policy last updated may 23 2018
1       ul sharing third party p share information thi...
2       service provider need access information perfo...
3       may also share information third party agreed ...
4       may give third party apps access account infor...
                              ...                        
3877    imdb right obligation monitor edit remove acti...
3878    however cooky allow take full advantage imdb e...
3879    notice term condition use change also use info...
3880                       last updated february 8 2018 b
3881    may e mail periodic reminder notice condition ...
Name: Sentence, Length: 3882, dtype: object

In [76]:
X_train.head()
y_train.head()
X_val.head()
y_val.head()

3432    0
1018    1
1941    1
3614    0
1200    0
Name: Point, dtype: int64

In [77]:
data.to_csv('out',index=True)

## Step 4: Building the Model using roBerta 

In [78]:
train_dataset =  pd.concat([X_train, y_train], axis=1)

In [79]:
from simpletransformers.classification import ClassificationModel


# Create a Roberta + distillation
args={'reprocess_input_data': True, 'max_seq_length': 300, 'num_train_epochs': 6, 'fp16': False, 'train_batch_size': 2, 'overwrite_output_dir': True}

roberta_model = ClassificationModel('roberta', 'distilroberta-base', num_labels=2, use_cuda=False, cuda_device=0, args=args)

# Train the model
roberta_model.train_model(train_dataset)

# Evaluate the model
result, model_outputs, wrong_predictions = roberta_model.eval_model(eval_dataset, acc=sklearn.metrics.f1_score)

Running loss: 0.033122


Running loss: 2.700990


Running loss: 0.017317


Running loss: 0.009077


Running loss: 0.007318


Running loss: 0.005761



In [80]:
eval_dataset = pd.concat([X_val, y_val], axis=1)
result, model_outputs, wrong_predictions = roberta_model.eval_model(eval_dataset, acc=sklearn.metrics.f1_score)

## Step 5: Evaluating the Model

In [81]:
with open('roberta_model.pickle','wb') as f:
    pickle.dump(roberta_model,f)

## Step 6: Saving the Trained Model Parameters

In [82]:
result

{'mcc': 0.5531309590678191,
 'tp': 281,
 'tn': 321,
 'fp': 109,
 'fn': 66,
 'acc': 0.7625508819538671,
 'eval_loss': 1.198574699209623}

In [83]:
from sklearn.metrics import confusion_matrix
import numpy as np
predict_class = np.argmax(model_outputs, axis=1)
predict_class = predict_class.tolist()

confusion_matrix(y_val, predict_class)

from sklearn.metrics import classification_report, f1_score

print("F1 Score = {:.2f}".format(f1_score(y_val, predict_class)))
print()
print(classification_report(y_val, predict_class))

F1 Score = 0.76

              precision    recall  f1-score   support

           0       0.83      0.75      0.79       430
           1       0.72      0.81      0.76       347

   micro avg       0.77      0.77      0.77       777
   macro avg       0.77      0.78      0.77       777
weighted avg       0.78      0.77      0.78       777

